<a href="https://colab.research.google.com/github/akalpit23/Interpretable-ML/blob/main/Assignment-4/Assignment_04_Interpretable_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04



## Akalpit Dawkhar

### **Interpretable ML-2**

### Overview

# 1. Importing neccesarry libraries, loading and preprocessing the data. 

In [35]:
!pip install collections
!pip show imodels

/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=85550) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
Name: imodels
Version: 1.4.6
Summary: Implementations of various interpretable models
Home-page: https://github.com/csinva/imodels
Author: Chandan Singh, Keyan Nasseri, Matthew Epland, Yan Shuo Tan, Omer Ronen, Tiffany Tang, Abhineet Agarwal, Theo Saarinen, Bin Yu, and others
Author-email: chandan_singh@berkeley.edu
License: 
Location: /Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/xai/lib/python3.12/site-packages
Requires: matplotlib, mlxtend, numpy, pandas, requests, scikit-learn, scipy, tqdm
Required-by: 


/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=85550) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [36]:
!pip install skope-rules shap lime scikit-learn seaborn skope-rules matplotlib setuptools six rulefit --upgrade
!pip install --upgrade imodels


/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=85550) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=85550) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import sklearn
import collections
import six
collections.Iterable = collections.abc.Iterable
sklearn.externals.six = six
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, precision_recall_curve, roc_auc_score
from sklearn.datasets import load_diabetes
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
import skrules.skope_rules
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import six
from imodels import  RuleFitRegressor, BoostedRulesClassifier, GreedyRuleListClassifier, GreedyTreeClassifier, GreedyTreeRegressor
from rulefit import RuleFit
from skrules import SkopeRules
from skrules import SkopeRules as SkopeRulesClassifier

In [20]:
def patched_init(*args, **kwargs):
    if 'base_estimator' in kwargs:
        kwargs['estimator'] = kwargs.pop('base_estimator')
    return original_init(*args, **kwargs)

original_init = skrules.skope_rules.BaggingClassifier.__init__
skrules.skope_rules.BaggingClassifier.__init__ = patched_init

In [24]:
def patch_estimator(cls):
    original_init = cls.__init__
    def patched_init(*args, **kwargs):
        if 'base_estimator' in kwargs:
            kwargs['estimator'] = kwargs.pop('base_estimator')
        return original_init(*args, **kwargs)
    cls.__init__ = patched_init

patch_estimator(skrules.skope_rules.BaggingClassifier)
patch_estimator(skrules.skope_rules.BaggingRegressor)

In [4]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Convert to DataFrame
diabetes_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)

# Print the first 5 rows
print(diabetes_df.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019907 -0.017646  
1 -0.039493 -0.068332 -0.092204  
2 -0.002592  0.002861 -0.025930  
3  0.034309  0.022688 -0.009362  
4 -0.002592 -0.031988 -0.046641  


In [6]:
# Check the shape of your DataFrame
print(X_train.shape)

# Check the column names of your DataFrame
print(diabetes_df.columns)

(353, 10)
Index(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], dtype='object')


# 2. RuleFit rule sit 

In [7]:
# Initialize the RuleFitRegressor
rf = RuleFitRegressor()

# Fit the model to your data
rf.fit(X_train, y_train)

RuleFitRegressor()

In [8]:
# Use the model to make predictions
predictions = rf.predict(X_test)

In [9]:
rule_df = rf.visualize()
rule_df

,rule,coef
2,bmi,40.10
3,bp,14.74
26,bp <= 0.02359,-5.82
24,bmi <= 0.07625 and s5 <= 0.022,-9.89
25,bp <= 0.01614,-1.72
18,bmi <= 0.07571 and s4 <= 0.05276 and s5 <= 0.02439,-8.71
21,bmi <= 0.0687 and s5 <= 0.01594,-1.85
23,bmi <= 0.01319 and s4 <= 0.05313,-1.09
20,bmi <= 0.1269 and bp <= 0.0035 and s4 <= 0.11918,-8.68
15,bmi <= 0.00942 and s3 > -0.0342,-13.42


In [10]:
# Calculate the mean absolute error
mae = mean_absolute_error(y_test, predictions)

# Calculate the mean squared error
mse = mean_squared_error(y_test, predictions)

# Calculate the R-squared score
r2 = r2_score(y_test, predictions)

# Print the metrics
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')

MAE: 43.75034173968487
MSE: 2832.482431566959
R2 Score: 0.46538291728620096


# 3. Greedy Rule List Classifier 

In [50]:
# Initialize and train GreedyRuleListClassifier
greedy_model = GreedyRuleListClassifier()
greedy_model.fit(X_train, y_train)
y_pred_greedy = greedy_model.predict(X_test)
accuracy_greedy = accuracy_score(y_test, y_pred_greedy)
print(f"GreedyRuleListClassifier Accuracy: {accuracy_greedy}")

GreedyRuleListClassifier Accuracy: 0.9122807017543859


# 4. Boosted Rules Classifier

In [51]:
# Initialize and train BoostedRulesClassifier
boosted_model = BoostedRulesClassifier()
boosted_model.fit(X_train, y_train)
y_pred_boosted = boosted_model.predict(X_test)
accuracy_boosted = accuracy_score(y_test, y_pred_boosted)
print(f"BoostedRulesClassifier Accuracy: {accuracy_boosted}")

BoostedRulesClassifier Accuracy: 0.956140350877193


/Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/xai/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


# 5. Greedy Tree Regressor 

In [53]:
model = GreedyTreeRegressor()  # or GreedyTreeRegressor() for regression tasks
model.fit(X_train, y_train)
accuracy_greedytree = accuracy_score(y_test, y_pred_boosted)
print(f"GreedyTreeRegressor: {accuracy_greedytree}")

GreedyTreeRegressor: 0.956140350877193


# 6. Refrences:


In [49]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from rulefit import RuleFit

# Initialize and train RuleFit
rulefit_model = RuleFit()
rulefit_model.fit(X_train, y_train)
y_pred_rf = rulefit_model.predict(X_test)
y_test_binary = np.where(y_test > np.median(y_test), 1, 0)
accuracy_rf = accuracy_score(y_test, y_pred_rf_binary)
print(f"RuleFit Accuracy: {accuracy_rf}")

# Initialize and train GreedyRuleListClassifier
greedy_model = GreedyRuleListClassifier()
greedy_model.fit(X_train, y_train)
y_pred_greedy = greedy_model.predict(X_test)
accuracy_greedy = accuracy_score(y_test, y_pred_greedy)
print(f"GreedyRuleListClassifier Accuracy: {accuracy_greedy}")

# Initialize and train BoostedRulesClassifier
boosted_model = BoostedRulesClassifier()
boosted_model.fit(X_train, y_train)
y_pred_boosted = boosted_model.predict(X_test)
accuracy_boosted = accuracy_score(y_test, y_pred_boosted)
print(f"BoostedRulesClassifier Accuracy: {accuracy_boosted}")

model = GreedyTreeRegressor()  # or GreedyTreeRegressor() for regression tasks
model.fit(X_train, y_train)

RuleFit Accuracy: 0.9649122807017544
GreedyRuleListClassifier Accuracy: 0.9122807017543859
BoostedRulesClassifier Accuracy: 0.956140350877193


/Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/xai/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.188e-02, tolerance: 1.062e-02
  model = cd_fast.enet_coordinate_descent(
/Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/xai/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
